In [3]:
import importlib

import wrapper_api.wrappers.communication_wrapper as cw
import wrapper_api.models.active_communication as ac

importlib.reload(ac)
importlib.reload(cw)

from wrapper_api.models.active_communication import ActiveCommunication
from wrapper_api.wrappers.communication_wrapper import CommunicationWrapper


ModuleNotFoundError: No module named 'Wrapper'